In [1]:
!pip install pymongo
# Dependencies
import requests
import json
import pandas as pd
import pymongo

In [2]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.marketdata

In [3]:
# Portfolio list and get the string list
# read from UI and insert into mongoDB 
symbol_dict = [{'symbol':'AAPL','active':'Y'},
               {'symbol':'TSLA','active':'Y'},
               {'symbol':'VOO','active':'Y'},
               {'symbol':'BABA','active':'Y'},
               {'symbol':'JPM','active':'Y'}]

In [4]:
# Insert a document into the 'portfolio' collection
# Read existing data from MongoDB
portfolio = db.portfolio.find()
is_data_found=db.portfolio.count()


data = pd.DataFrame(list(portfolio))
data
print(is_data_found)

# from new list above
for i in range(len(symbol_dict)):
    #print(list_index[i]['symbol'])
    cnt=0
    # avoid duplicate check if symbol is already active in DB
    if is_data_found !=0:
        cnt=data[(data['active']==symbol_dict[i]['active']) & (data['symbol']==symbol_dict[i]['symbol'])]['symbol'].count()
    else:
        # first record
        cnt=1
        
    if (cnt!=0 | is_data_found==0):
        db.portfolio.insert_one(symbol_dict[i])

0


C:\Users\gecha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  after removing the cwd from sys.path.


In [5]:
symbol_df = pd.DataFrame.from_dict(db.portfolio.find())
#symbol_df=symbol_df.append({'symbol':'TSLA','active':'Y'},ignore_index=TRUE)
active_symbol= list(symbol_df[symbol_df['active']=='Y']['symbol'])  
symbol_str=''
for i in range(len(active_symbol)):
    if i == len(active_symbol)-1:
        symbol_str = symbol_str + active_symbol[i] 
    else:
        symbol_str = symbol_str + active_symbol[i] + '%2C'

symbol_str

'AAPL%2CTSLA%2CVOO%2CBABA%2CJPM'

In [6]:
# URL for GET requests to retrieve symbol data
# append the active symbol from above list
url = "https://marketdata.websol.barchart.com/getQuote.json?apikey=0b6bf035918d6d5e9f4b803bded9a20f&fields=fiftyTwoWkHigh%2CfiftyTwoWkHighDate%2CfiftyTwoWkLow%2CfiftyTwoWkLowDate%2Ctick&symbols="+symbol_str



In [7]:
# Pretty print JSON for all launchpads
response = requests.get(url)
response_json = response.json() 

In [8]:
# rearrange dataset
#df[['symbol','close']]
#new_df = df[['symbol','name','open','lastPrice','netChange','tradeTimestamp',
#             'fiftyTwoWkHigh','fiftyTwoWkHighDate','fiftyTwoWkLow','fiftyTwoWkLowDate','serverTimestamp']]
#new_df

In [9]:
# using db = client.marketdata

# Query all marketdata
# Here, get the daily index price to dataframe
index_price = db.index_price.find()
is_data_found=db.index_price.count()

#list_index=response_json["results"] 
if is_data_found !=0:
    data = pd.DataFrame(list(index_price))
    data
print(is_data_found)
data

0


C:\Users\gecha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


""


In [10]:
# Iterate through each indexprice in the collection
# insert into DB for index price

# Insert a document into the 'indexprice' collection
list_index=response_json["results"] 
#len(list_index)
for i in range(len(list_index)):
    #print(list_index[i]['symbol'])
    cnt=0
    #print(list_index[i]['serverTimestamp'])
    #print(data['tradeTimestamp']==list_index[i]['tradeTimestamp'])
    # avoid duplicate
    #if is_data_found !=0:
    #    cnt=data[(data['tradeTimestamp']==list_index[i]['tradeTimestamp']) & (data['symbol']==list_index[i]['symbol'])]['symbol'].count()
        # if cnt is zero then its a new record
        #print(cnt)
    #    if (cnt == 0):
    #       cnt=1 
    #else:
        # first record
    #    cnt=1
        
    #if record exists ignore the exception
    if (cnt!=0) | (is_data_found==0):
        try:
            db.index_price.insert_one(list_index[i])
        except Exception:
            pass
    
print("Total number of records in index_list in database is:", db.index_price.count())
        

Total number of records in index_list in database is: 5


C:\Users\gecha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


In [11]:
# Iterate through each symbol in the collection, uncomment the code to see the results
#index_price = db.index_price.find()
#for ind in index_price:
#len(list_index)
#print(is_data_found)
#list_index[0]['serverTimestamp']
#    print(ind)
#for i in range(len(index_price)):
#    print(index_price[i])

 

In [12]:
# TO populate the history record read through individual symbol and gather the data in list
symbol_df = pd.DataFrame.from_dict(db.portfolio.find()) 
active_symbol=[]
active_symbol= list(symbol_df[symbol_df['active']=='Y']['symbol'])  
#symbol_str=''

hist_list_index=[]
hist_index=[]

hist_url= "https://marketdata.websol.barchart.com/getHistory.json?apikey=0b6bf035918d6d5e9f4b803bded9a20f&type=minutes&startDate=20190801&endDate=20191011&maxRecords=100&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration&symbol="

for i in range(len(active_symbol)):
    hist_response = requests.get(hist_url+active_symbol[i])
    #print(hist_url+active_symbol[i])
    hist_response_json = hist_response.json()
    hist_index = hist_response_json["results"]
    for i in range(len(hist_index)):
        hist_list_index.append(hist_index[i])



#len(hist_list_index)
#len(hist_list_index[0])

#for i in range(len(hist_list_index)):
#    print(hist_list_index[i])

In [13]:
# Iterate through each history indexprice in the collection
# insert into DB for history ndex price

# Insert a document into the 'indexprice' collection
# Query all marketdata
# Here, get the daily index price to dataframe
index_price = db.index_price_hist.find()
is_data_found=db.index_price_hist.count()

 
data = pd.DataFrame(list(index_price))
data
print(is_data_found)

# use hist_list_index from above
for i in range(len(hist_list_index)):
    #print(list_index[i])
    #db.index_price_hist.insert_one(hist_list_index[i])
        #print(list_index[i]['symbol'])
    #cnt=0
    # avoid duplicate
    # check if the symbol and systemTimestamp record is found
    #if is_data_found !=0:
    #    cnt=data[(data['timestamp']==hist_list_index[i]['timestamp']) & (data['symbol']==hist_list_index[i]['symbol'])]['symbol'].count()
    # else:
        # first record
        # cnt=1
        
    #if record exists ignore the exception
    
    try:
        db.index_price_hist.insert_one(hist_list_index[i])
    except Exception:
        pass
        
    #if (cnt!=0 | is_data_found==0):
        #db.index_price_hist.insert_one(hist_list_index[i])

C:\Users\gecha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


0


In [14]:
# Iterate through each symbol in the collection, uncomment the code to see the results
#index_price = db.index_price.find()
#index_price = db.index_price_hist.find()
#for ind in index_price:
#    print(ind)
db.index_price_hist.count()

C:\Users\gecha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


500

In [15]:


#list_index=response_json["results"]
index_price = db.index_price.find()
data = pd.DataFrame(list(index_price))
 

    
dict_data = data.to_dict
dict_data

<bound method DataFrame.to_dict of                         _id   close dayCode exchange  fiftyTwoWkHigh  \
0  5e17b93987822f924cb8dc8c  309.63       9   NASDAQ          310.43   
1  5e17b93987822f924cb8dc8d  481.34       9   NASDAQ          498.80   
2  5e17b93987822f924cb8dc8e  300.12       9     AMEX          300.19   
3  5e17b93987822f924cb8dc8f  221.78       9     NYSE          223.08   
4  5e17b93987822f924cb8dc90  137.44       9     NYSE          141.10   

  fiftyTwoWkHighDate  fiftyTwoWkLow fiftyTwoWkLowDate flag    high  ...  \
0         2020-01-09         149.22        2019-01-14    s  310.43  ...   
1         2020-01-09         176.99        2019-06-03    s  498.80  ...   
2         2020-01-09         234.85        2019-01-10    s  300.19  ...   
3         2020-01-09         146.54        2019-01-14    s  223.08  ...   
4         2020-01-02          98.09        2019-03-25    s  138.23  ...   

                        name  netChange    open percentChange  \
0               